# Key Points from the Description
### 1. Objective of the Challenge:

- The primary objective is to forecast traffic volume (DLThpVol) at the beam level using a multivariate time series forecasting model.
- The task is to leverage state-of-the-art (SOTA) forecasting machine learning models to handle high-resolution data that captures the intricacies of real-world traffic scenarios.


### 2. Dataset Details:

- Four datasets are provided, each corresponding to a different network performance metric:
    - `traffic_DLThpVol.csv`: Throughput volume (target variable for forecasting).
    - `traffic_DLThpTime.csv`: Throughput time.
    - `traffic_DLPRB.csv`: Physical Resource Block (PRB) utilization.
    - `traffic_MR_number.csv`: User count.
- The datasets cover 2,880 beams across 30 base stations with 3 cells and 32 beams each. Data is recorded hourly for a five-week period.


In [ ]:
import numpy as np
import pandas as pd
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.metrics import mean_absolute_error
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from lightgbm import LGBMRegressor
from sklearn.model_selection import TimeSeriesSplit
import matplotlib.pyplot as plt
import random
from math import pi
import statsmodels.api as sm
plt.style.use('fivethirtyeight')
from statsmodels.tsa.seasonal import seasonal_decompose


pd.set_option('display.max_rows', 5)
pd.set_option('display.max_columns', 1000)

In [ ]:
#chatGPT Generated the docs and made the code much cleaner, sorry if it's hallucinated
DATA_PATH = "/Users/ahmed/Downloads/computerscience/AI-Compeitions/Spatio-Temporal-Beam-Level-Traffic-Forecasting/data/"
SUBMISSION_FILE = "enhanced_submission.csv"
N_BASE = 30
N_CELL = 3
N_BEAM = 32
RANDOM_STATE = 123
TRAIN_RATIO = 0.8  # Ratio of data used for training

# Load data
def load_data():
    """
    Load the sample submission and target data from CSV files.
    """
    ss = pd.read_csv(DATA_PATH + "SampleSubmission.csv")
    ty = pd.read_csv(DATA_PATH + "traffic_DLThpVol.csv")
    return ss, ty

# Feature Engineering
def create_features(df):
    """
    Generate features based on datetime information.
    
    Parameters:
    df (DataFrame): The input dataframe for which to generate features.
    
    Returns:
    DataFrame: A dataframe containing the generated features.
    """
    df['hour'] = df.index % 24
    df['day_of_week'] = (df.index // 24) % 7
    df['day'] = df.index // 24
    
    return df

# Model Training, Prediction, and Validation
def train_and_validate(x_train, x_val, y_train, y_val, ss):
    """
    Train the model, validate it, and predict the results for each base, cell, and beam combination.
    
    Parameters:
    x_train (DataFrame): Training features.
    x_val (DataFrame): Validation features.
    y_train (DataFrame): Training target values.
    y_val (DataFrame): Validation target values.
    ss (DataFrame): The sample submission dataframe.
    
    Returns:
    DataFrame: The updated sample submission with predictions.
    """
    rs = RANDOM_STATE
    val_scores = []

    for base in range(N_BASE):
        for cell in range(N_CELL):
            for beam in range(N_BEAM):
                rs += 1
                mod_col = f"{base}_{cell}_{beam}"
                
                # Initialize and train model
                model = HistGradientBoostingRegressor(random_state=rs, loss="absolute_error")
                model.fit(x_train, y_train[mod_col])
                
                # Validate model
                val_pred = np.clip(model.predict(x_val), 0, 255)
                val_score = mean_absolute_error(y_val[mod_col], val_pred)
                val_scores.append(val_score)
                
                # Predict and clip predictions to the expected range
                pred = np.clip(model.predict(x_val), 0, 255)
                
                # Populate the sample submission with predictions
                for k in range(168):
                    ss.at[f"traffic_DLThpVol_test_5w-6w_{k}_{mod_col}", "Target"] = pred[k]
                    ss.at[f"traffic_DLThpVol_test_10w-11w_{168-k-1}_{mod_col}", "Target"] = pred[-k-1]
    
    avg_val_score = np.mean(val_scores)
    print(f"Average Validation MAE: {avg_val_score:.4f}")
    
    return ss, avg_val_score

# Main Execution
def main():
    """
    Main function to execute the model training, validation, and submission generation.
    """
    # Load data
    ss, ty = load_data()
    
    # Determine training and validation size
    train_size = int(len(ty) * TRAIN_RATIO)
    
    # Feature engineering
    x_train = create_features(pd.DataFrame(index=np.arange(train_size)))
    x_val = create_features(pd.DataFrame(index=np.arange(train_size, len(ty))))
    x_test = create_features(pd.DataFrame(index=np.arange(len(ty) + 1008))).iloc[len(ty):]
    
    # Prepare sample submission
    ss = ss.set_index("ID", drop=False)
    ss["Target"] = ss["Target"].astype("float16")
    
    # Split target data into training and validation sets
    y_train = ty.iloc[:train_size]
    y_val = ty.iloc[train_size:]
    
    # Train models, validate, and generate predictions
    ss, avg_val_score = train_and_validate(x_train, x_val, y_train, y_val, ss)
    
    # Save submission
    ss.to_csv(SUBMISSION_FILE, index=False)
    print(f"Submission saved to {SUBMISSION_FILE} with an average validation MAE of {avg_val_score:.4f}")

# Run the main function
if __name__ == "__main__":
    main()

/var/folders/4c/h16036bx2ync7wqkqvrl2p1h0000gn/T/ipykernel_7733/3689726404.py:79: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.1947424348546613' has dtype incompatible with float16, please explicitly cast to a compatible dtype first.
  ss.at[f"traffic_DLThpVol_test_5w-6w_{k}_{mod_col}", "Target"] = pred[k]


Average Validation MAE: 0.2001
Submission saved to enhanced_submission.csv with an average validation MAE of 0.2001


In [ ]:
from manim import *

class TrainAndPredict(Scene):
    def construct(self):
        # Title
        title = Text("Model Training and Prediction", font_size=36)
        self.play(Write(title))
        self.wait(1)
        self.play(title.animate.to_edge(UP))

        # Display the loop structure
        base_loop = Text("for base in range(N_BASE):", font_size=28)
        cell_loop = Text("for cell in range(N_CELL):", font_size=28).next_to(base_loop, DOWN, aligned_edge=LEFT)
        beam_loop = Text("for beam in range(N_BEAM):", font_size=28).next_to(cell_loop, DOWN, aligned_edge=LEFT)
        
        self.play(Write(base_loop))
        self.play(Write(cell_loop))
        self.play(Write(beam_loop))
        self.wait(1)

        # Highlight model initialization and training
        model_init_train = Text("Initialize and train model", font_size=24).next_to(beam_loop, DOWN, aligned_edge=LEFT)
        model_predict = Text("Predict and clip predictions", font_size=24).next_to(model_init_train, DOWN, aligned_edge=LEFT)
        populate_ss = Text("Populate sample submission", font_size=24).next_to(model_predict, DOWN, aligned_edge=LEFT)
        
        self.play(Write(model_init_train))
        self.play(Write(model_predict))
        self.play(Write(populate_ss))
        self.wait(1)

        # Visualize the training and prediction process
        base_box = SurroundingRectangle(base_loop, color=BLUE)
        cell_box = SurroundingRectangle(cell_loop, color=GREEN)
        beam_box = SurroundingRectangle(beam_loop, color=YELLOW)

        self.play(Create(base_box))
        self.play(ReplacementTransform(base_box, cell_box))
        self.play(ReplacementTransform(cell_box, beam_box))
        self.wait(1)
        
        # Highlight model initialization, training, prediction, and submission update
        model_box = SurroundingRectangle(VGroup(model_init_train, model_predict, populate_ss), color=RED)
        self.play(ReplacementTransform(beam_box, model_box))
        self.wait(1)

        # Emphasize the completion of one iteration
        complete_iteration = Text("One iteration completed", font_size=24, color=ORANGE).to_edge(DOWN)
        self.play(Write(complete_iteration))
        self.wait(2)

        # Conclude
        self.play(FadeOut(VGroup(base_loop, cell_loop, beam_loop, model_init_train, model_predict, populate_ss, model_box, complete_iteration)))
        self.play(Write(Text("End of Function", font_size=36).move_to(ORIGIN)))
        self.wait(2)

# To render the scene, use the following command in your terminal:
# manim -pql <script_name>.py TrainAndPredict

In [11]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from lightgbm import LGBMRegressor
from sklearn.model_selection import TimeSeriesSplit
import matplotlib.pyplot as plt
import random
from math import pi
import statsmodels.api as sm
plt.style.use('fivethirtyeight')
from statsmodels.tsa.seasonal import seasonal_decompose

# Reading Data

In [3]:
def reading_data(path:str)-> pd.DataFrame:
    sample = pd.read_csv(path+"SampleSubmission.csv")
    volume = pd.read_csv(path + "traffic_DLThpVol.csv")
    time = pd.read_csv(path + "traffic_DLThpTime.csv")
    PRB = pd.read_csv(path + "traffic_DLPRB.csv")
    user_count = pd.read_csv(path + "traffic_MR_number.csv")
    return sample, volume, time, PRB, user_count

sample, volume, time, PRB, user_count = reading_data("/Users/ahmed/Downloads/computerscience/AI-Compeitions/Spatio-Temporal-Beam-Level-Traffic-Forecasting/data/")
#train = reduce_memory_usage(df = train,categ = True)
#test = reduce_memory_usage(df = train,categ = True)

In [74]:
vol_transposed = volume.T
vol_transposed.columns = vol_transposed.columns
vol_transposed = vol_transposed.drop(vol_transposed.index[0])
vol_transposed

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511,512,513,514,515,516,517,518,519,520,521,522,523,524,525,526,527,528,529,530,531,532,533,534,535,536,537,538,539,540,541,542,543,544,545,546,547,548,549,550,551,552,553,554,555,556,557,558,559,560,561,562,563,564,565,566,567,568,569,570,571,572,573,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,589,590,591,592,593,594,595,596,597,598,599,600,601,602,603,604,605,606,607,608,609,610,611,612,613,614,615,616,617,618,619,620,621,622,623,624,625,626,627,628,629,630,631,632,633,634,635,636,637,638,639,640,641,642,643,644,645,646,647,648,649,650,651,652,653,654,655,656,657,658,659,660,661,662,663,664,665,666,667,668,669,670,671,672,673,674,675,676,677,678,679,680,681,682,683,684,685,686,687,688,689,690,691,692,693,694,695,696,697,698,699,700,701,702,703,704,705,706,707,708,709,710,711,712,713,714,715,716,717,718,719,720,721,722,723,724,725,726,727,728,729,730,731,732,733,734,735,736,737,738,739,740,741,742,743,744,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767,768,769,770,771,772,773,774,775,776,777,778,779,780,781,782,783,784,785,786,787,788,789,790,791,792,793,794,795,796,797,798,799,800,801,802,803,804,805,806,807,808,809,810,811,812,813,814,815,816,817,818,819,820,821,822,823,824,825,826,827,828,829,830,831,832,833,834,835,836,837,838,839
0_0_0,0.0,0.0,0.0,0.000000,0.445035,0.253590,0.465449,0.000000,0.000000,0.060065,0.715016,0.022865,0.329495,1.370149,0.000000,0.318568,0.101982,0.755171,0.062647,0.966872,0.513887,2.237365,1.363147,0.544219,2.175297,0.284434,0.000000,0.180691,0.132375,0.000000,0.000000,0.085466,0.148006,1.884579,0.293084,0.148146,0.291175,0.989395,0.210489,0.531997,0.600646,0.890992,0.531791,0.300827,0.077618,1.323744,0.857099,0.556703,1.374687,0.193978,0.006657,0.049015,0.000000,0.390330,0.000000,0.130603,0.0,0.000000,0.204602,0.154244,0.494582,0.201253,0.238395,0.250607,0.027774,0.450031,0.713835,0.878292,0.349324,0.199062,0.339059,0.566022,0.464514,0.391975,0.000000,0.195073,0.119431,0.000000,0.000000,0.137873,0.000000,0.294343,0.109141,0.333489,0.00000

In [77]:
time.T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511,512,513,514,515,516,517,518,519,520,521,522,523,524,525,526,527,528,529,530,531,532,533,534,535,536,537,538,539,540,541,542,543,544,545,546,547,548,549,550,551,552,553,554,555,556,557,558,559,560,561,562,563,564,565,566,567,568,569,570,571,572,573,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,589,590,591,592,593,594,595,596,597,598,599,600,601,602,603,604,605,606,607,608,609,610,611,612,613,614,615,616,617,618,619,620,621,622,623,624,625,626,627,628,629,630,631,632,633,634,635,636,637,638,639,640,641,642,643,644,645,646,647,648,649,650,651,652,653,654,655,656,657,658,659,660,661,662,663,664,665,666,667,668,669,670,671,672,673,674,675,676,677,678,679,680,681,682,683,684,685,686,687,688,689,690,691,692,693,694,695,696,697,698,699,700,701,702,703,704,705,706,707,708,709,710,711,712,713,714,715,716,717,718,719,720,721,722,723,724,725,726,727,728,729,730,731,732,733,734,735,736,737,738,739,740,741,742,743,744,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767,768,769,770,771,772,773,774,775,776,777,778,779,780,781,782,783,784,785,786,787,788,789,790,791,792,793,794,795,796,797,798,799,800,801,802,803,804,805,806,807,808,809,810,811,812,813,814,815,816,817,818,819,820,821,822,823,824,825,826,827,828,829,830,831,832,833,834,835,836,837,838,839
Unnamed: 0,0.0,1.000000,2.000000,3.000000,4.0,5.000000,6.000000,7.0,8.000000,9.000000,10.000000,11.000000,12.000000,13.000000,14.000000,15.0,16.000000,17.000000,18.000000,19.000000,20.000000,21.000000,22.000000,23.000000,24.000000,25.000000,26.000000,27.00000,28.000000,29.000000,30.0,31.000000,32.000000,33.000000,34.000000,35.000000,36.000000,37.000000,38.000000,39.000000,40.000000,41.000000,42.000000,43.000000,44.000000,45.000000,46.000000,47.000000,48.000000,49.000000,50.000000,51.000000,52.000000,53.000000,54.000000,55.000000,56.000000,57.000000,58.000000,59.00000,60.000000,61.000000,62.000000,63.000000,64.000000,65.000000,66.000000,67.000000,68.000000,69.000000,70.000000,71.000000,72.000000,73.000000,74.000000,75.000000,76.000000,77.00

In [80]:
time_transposed = time.T
time_transposed = time_transposed.drop(time_transposed.index[0])
time_transposed

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511,512,513,514,515,516,517,518,519,520,521,522,523,524,525,526,527,528,529,530,531,532,533,534,535,536,537,538,539,540,541,542,543,544,545,546,547,548,549,550,551,552,553,554,555,556,557,558,559,560,561,562,563,564,565,566,567,568,569,570,571,572,573,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,589,590,591,592,593,594,595,596,597,598,599,600,601,602,603,604,605,606,607,608,609,610,611,612,613,614,615,616,617,618,619,620,621,622,623,624,625,626,627,628,629,630,631,632,633,634,635,636,637,638,639,640,641,642,643,644,645,646,647,648,649,650,651,652,653,654,655,656,657,658,659,660,661,662,663,664,665,666,667,668,669,670,671,672,673,674,675,676,677,678,679,680,681,682,683,684,685,686,687,688,689,690,691,692,693,694,695,696,697,698,699,700,701,702,703,704,705,706,707,708,709,710,711,712,713,714,715,716,717,718,719,720,721,722,723,724,725,726,727,728,729,730,731,732,733,734,735,736,737,738,739,740,741,742,743,744,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767,768,769,770,771,772,773,774,775,776,777,778,779,780,781,782,783,784,785,786,787,788,789,790,791,792,793,794,795,796,797,798,799,800,801,802,803,804,805,806,807,808,809,810,811,812,813,814,815,816,817,818,819,820,821,822,823,824,825,826,827,828,829,830,831,832,833,834,835,836,837,838,839
0_0_0,0.00000,0.000000,0.014916,0.000000,0.0,0.013926,0.321226,0.000000,0.084925,0.036893,0.642004,0.171075,0.189675,0.704637,0.131426,0.000000,0.000000,0.573367,0.008087,1.467203,1.701504,1.616717,0.679566,0.526874,1.465043,0.362747,0.000000,0.174570,0.087691,0.074761,0.000000,0.000000,0.300773,1.903020,0.031982,0.393027,0.000000,0.667465,0.200895,0.372803,0.940979,0.838185,0.376186,0.154012,0.704551,1.503181,0.882970,0.252628,0.902917,0.238305,0.135669,0.326141,0.066527,0.054085,0.163094,0.000000,0.094356,0.050302,0.164430,0.00000,0.548692,0.342359,0.435901,0.364008,0.255652,0.314188,0.365846,0.758748,0.493769,0.280106,0.286040,0.269331,0.630915,0.540544,0.184381,0.000000,0.028710,0.031728,0.000000,0.246442,0.000000,0.163730,0.000000,0.4

### Submission Requirements:

- Participants must submit a CSV file named traffic_forecast.csv containing forecasted traffic volume (DLThpVol) for the designated weeks.
- The submission should include forecasted values for the two target weeks (6th week and 11th week) at an hourly granularity for all combinations of hour, gnodeb, cell, and beam.
- The format for the identifiers in the submission file is:
- `traffic_DLThpVol_test_5w-6w_hour_gnodeb_cell_beam`
- `traffic_DLThpVol_test_10w-11w_hour_gnodeb_cell_beam`